In [1]:
import numpy as np
from skimage import io
from skimage import exposure
from skimage import color

import supervisely_lib as sly

import os

from skimage.transform import rescale

In [2]:
# Change this field to the name of your team, where target workspace exists.
team_name = "marcwang17" # Automatically inserted

# Change this field to the of your workspace, where target project exists.
workspace_name = "First Workspace" # Automatically inserted

# If you are running this notebook on a Supervisely web instance, the connection
# Edit those values if you run this notebook on your own PC
# details below will be filled in from environment variables automatically.
#
# If you are running this notebook locally on your own machine, edit to fill in the
# connection details manually. You can find your access token at
# "Your name on the top right" -> "Account settings" -> "API token".
address = os.environ['SERVER_ADDRESS']
token = os.environ['API_TOKEN']

In [3]:
# Initialize API object
api = sly.Api(address, token)

In [4]:
project_name = 'split_dt_ups'

In [5]:
# Get IDs of team and workspace
team = api.team.get_info_by_name(team_name)
workspace = api.workspace.get_info_by_name(team.id, workspace_name)

In [6]:

project = api.project.get_info_by_name(workspace.id, project_name)
if project is None:
    project = api.project.create(workspace.id, project_name, change_name_if_conflict=True)



In [7]:
images_dir = './data'

In [8]:
img_paths = sly.fs.list_files(images_dir)

In [9]:

for img_path in img_paths:
    full_im = io.imread(img_path)
    file_name = sly.fs.get_file_name(img_path)
    dataset = api.dataset.create(project.id, file_name, change_name_if_conflict=True)
    
    ls_name = []
    ls_im = []
    
    for i in range(full_im.shape[0]):
        sub_im = full_im[i,...]
        upscaled = rescale(sub_im, scale = 4)
        rescaled_im = exposure.rescale_intensity(upscaled, in_range='image', out_range=np.uint8)
        rgb_im = color.gray2rgb(rescaled_im)
        
        ls_name.append('{}_{}.png'.format(file_name, i))
        ls_im.append(rgb_im)
        
    api.image.upload_nps(dataset.id, ls_name, ls_im)

/usr/local/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
